# Event Time
This is an important topic to cover discretely because spark's Dstream API does not support processing information with respect to event time. At a higher level, in stream processing systems there are effectively two relevant times for each event: the time at which it actually occurred (event time), and the time that it was processes or reached the stream processing system (processing time)

**Event Time**
This is the time that is embedded in the data itself. It is most often, though not required to be, the time that an event actually occurs. Thgis is important to use because it provides a more robust way of comparing events against one another. The challenge here is that event data can be late or out of order. This means that the stream processing system must be able to handle out-of-order or late data

**Processing Time**
This is the time at which the stream-processing system actually received data. This is usually less important than event time because when it's processed is largely an implementation detail

# Stateful Processing
This is only necessary when you need to use or update intermediate information (state) over longer periods of time (in either a microbatch or a record-at-a-time approach). This can happen when you are using event time or when you are performing an aggregation on a key, whether that involves event time or not

# Arbitrary Stateful Processing
There are times when you need fine-grained control over what state should be stored, how it is updated, and when it should be removed, either explicitly or via a time-out. This is called arbitrary stateful processing and spark allows you to essentially store whatever information you like over the course of the processing of a stream. This provides immense flexibility and power and allows for some complex business logic to be handled quite easily. Here are some examples:
* You'd like to record information about user sessions on an ecommerce site. For instance, you might want to track what pages users visit over the course of this session in order to provide recommendations in real time during their next session. Naturally, these sessions have completely arbitrary start and stop times that are unique to that user
* Your company would like to report on errors in the web application but only if five events occur during a user's session. You could do this with count-based windows that only emit a result if five events of some type occur
* You'd like to deduplicate records over time. To do so, you're going to need to keep track of every record that you see before deduplicating it.

# Event-Time Basics

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/11/09 10:34:06 WARN Utils: Your hostname, kevin resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
22/11/09 10:34:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/09 10:34:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set('spark.sql.shuffle.partitions', '5')
static = spark.read.json('/home/kevin/Desktop/Big-Data-with-Pyspark/data/activity-data')
streaming = spark\
                .readStream\
                .schema(static.schema)\
                .option('maxFilesPerTrigger', 10)\
                .json('/home/kevin/Desktop/Big-Data-with-Pyspark/data/activity-data')

streaming.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In this dataset, there are two time-based columns. The creation_time column defines when an event was created, whereas the arrival_time defines when an event hit our servers somewhere upstream. 

# Windows on Event Time
The first step in event-time analysis is to convert the timestamp column into the proper spark SQL timestamp type. Our current column is unixtime nanoseconds (represented as long), therefore we're going to manipulate it

In [3]:
withEventTime = streaming.selectExpr(
    "*",
    "cast(cast(Creation_time as double)/1000000000 as timestamp) as event_time"
)


## Tumbling Windows
The simplest operation is simply to count the number of occurrences of an event in a given window. This depicts the process when performing a simple summation based on the input data and a key 



<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/07_tumbling_windows.png">